In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import urllib.request 
from bs4 import BeautifulSoup as bs 
import re 
import json, csv 

def toJson(recipe_dict): 
	with open('recipe.json', 'w', encoding='utf-8') as file : 
		json.dump(recipe_dict, file, ensure_ascii=False, indent='\t') 

def toCSV(recipe_list): 
	with open('ingredients.csv', 'w', encoding='utf-8', newline='') as file : 
	  csvfile = csv.writer(file) 
	for row in recipe_list: 
		#csvfile.writerow(row) 

def url_func(n, m): 
	num_range = range(n,m) 
	url = "https://www.10000recipe.com/profile/recipe.html?uid=gdubu33&page=" 
	url_list = [] 

	for num in num_range: 
		req = urllib.request.Request(url + str(num)) # page 수 붙이기 
		code = urllib.request.urlopen(url).read() 
		soup = bs(code, "html.parser")

		try: 
			res = soup.find(class_='cont_list st3') 
			for i in res.find_all('a'): 
				url_tmp = i.get('href') 
				url_list.append(url_tmp) 
		except(AttributeError): 
			pass 
	return url_list 

# 사이트마다 메뉴 이름, 재료, 레시피 등등 크롤링 후 
# json 형식으로 저장 

num_id=0 
food_dicts = [] 
ingre_set = set() 

# 재료 목록들을 담기 위한 

url_lists = url_func(1,165) 

for url_str in url_lists: 
	url = "https://www.10000recipe.com" 
	url = url + url_str 
	req = urllib.request.Request(url) 
	code = urllib.request.urlopen(url).read() 
	soup = bs(code, "html.parser") 

	info_dict = {} 
	ingre_list = [] 
	ingre_dict = {} 
	recipe_list = [] 
	recipe_dict = {} 
	food_dict = {} 

	# menu_name 
	res = soup.find('div', 'view2_summary') 
	res = res.find('h3') 
	menu_name = res.get_text() 

	# menu_img 
	res = soup.find('div', 'centeredcrop') 
	res = res.find('img') 
	menu_img = res.get('src') 

	# menu_summary 
	res = soup.find('div', 'view2_summary_in') 
	menu_summary = res.get_text().replace('\n','').strip() 
	
	# menu_info 
	res = soup.find('span', 'view2_summary_info1') # menu_info_1 
	menu_info_1 = res.get_text() 
	res = soup.find('span', 'view2_summary_info2') # menu_info_2 
	menu_info_2 = res.get_text() 
	res = soup.find('span', 'view2_summary_info3') # menu_info_3 
	menu_info_3 = res.get_text() 
	
	# info dict 
	info_dict = {"info1":menu_info_1, "info2":menu_info_2, "info3":menu_info_3} 
	# ingredient 
	res = soup.find('div','ready_ingre3') 
	try : 
		for n in res.find_all('ul'): 
			for tmp in n.find_all('li'): 
				ingredient_name = tmp.get_text().replace('\n','').replace(' ','') 
				count = tmp.find('span') 
				ingredient_tmp = count.get_text() 
				ingredient_name = re.sub(ingredient_tmp, '', ingredient_name) # ingredient_name 
				ingredient_unit = ingredient_tmp.replace('/','').replace('+','') 
				ingredient_unit = ''.join([i for i in ingredient_unit if not i.isdigit()]) 
				# ingredient_unit 
				ingredient_count = re.sub(ingredient_unit, '', ingredient_tmp) 
				# ingredient_count 
				# ingre_list 
				ingre_dict = {"ingre_name":ingredient_name, 
							"ingre_count":ingredient_count, 
							"ingre_unit":ingredient_unit,} 
				ingre_list.append(ingre_dict) 

				# set에 업데이트 
				ingre_set.add(ingredient_name) 
	except(AttributeError): 
		pass 

	# recipe 
	res = soup.find('div','view_step') 
	for n in res.find_all('div','view_step_cont'): 
		recipe_step_txt = n.get_text().replace('\n',' ')
		tmp = n.find('img') 
		recipe_step_img = tmp.get('src') 
		# recipe_list 
		recipe_dict = {"txt":recipe_step_txt, 
					"img":recipe_step_img,} 
		recipe_list.append(recipe_dict) 
	# 재료 형식에 맞지 않게 올라온 글들 skip 
	if not ingre_list: 
		continue 
	
	num_id = num_id + 1 
	food_dict = {"id":num_id, 
				"name":menu_name, 
				"img":menu_img, 
				"summary":menu_summary, 
				"info":info_dict, "ingre":ingre_list, 
				"recipe":recipe_list} 
	food_dicts.append(food_dict) 
	
# json 생성 
toJson(food_dicts) 

# ingredients list csv 생성 
ingre_list_csv=[] 
for i in ingre_set: 
	tmp_l = [] 
	tmp_l.append(i) 
	ingre_list_csv.append(tmp_l) 
 
toCSV(ingre_list_csv)



ValueError: ignored

In [10]:
import json, os, shutil, time, requests
from bs4 import BeautifulSoup
import re

def thumb_png_get(pic):
    # pic_url = pic.find_all("img")[0]["src"]
    pic_url = pic.find("div").find_all("img")[0]["src"]
    href_path = pic.find("div").a["href"]
    if "_m.jpg" in pic_url:
        print(pic_url, href_path)
        create_picture(pic_url)
        create_json(href_path)
    else:
        print("Not Thumb_png")
        pass
    # create_picture(pic_url)
    # create_json(href_path)

def create_json(href_path):
    dir_json = 'data/json/'
    if os.path.exists(dir_json):
        with open('data/json/%s.json' % (keyword), 'a', encoding="utf-8") as out_file:
            url_deep = "https://www.10000recipe.com/recipe/view.html?seq=%s" % str(href_path).split("/")[2]
            print(url_deep)
            data = BeautifulSoup(requests.get(url_deep).text, "html.parser")
            if data.select_one("#contents_area > div.view_cont > div") != None:
                print("cont_ingre")
                print(data.select_one("#contents_area > div.view_cont > div").find("dd").text)
                pass
            elif data.select_one("#contents_area > div.cont_ingre2") != None:
                print("cont_ingre2")
                print([{str(x.get_text()).split("                                                        ")[0] : x.span.text} 
                       for x in data.select_one("#divConfirmedMaterialArea > ul:nth-child(1)").find_all("li")])
                out_file.write(json.dumps({num: [
                    str(x.get_text()).split("                                                        ")[0] for x in
                    data.select_one("#divConfirmedMaterialArea > ul:nth-child(1)").find_all("li")]},
                    ensure_ascii=False) + "\n")
            else:
                print("?????")
                pass
    else:
        os.makedirs(dir_json)


def create_picture(pic_url):
    response = requests.get(pic_url, stream=True)
    dir_pic = 'data/picture/%s' % keyword
    if os.path.exists(dir_pic):
        with open('data/picture/%s/%s.png' % (keyword, str(num).zfill(5)), 'wb') as out_file:
            shutil.copyfileobj(response.raw, out_file)
        # del response
    else:
        os.makedirs(dir_pic)

if __name__ == '__main__':
    keywords = ["콩나물국밥"]
    for keyword in keywords:
        page = 1
        num = 1
        while True:
            try:
                url = "https://www.10000recipe.com/recipe/list.html?q=%s&order=reco&page=%s" % (keyword, page)
                print(url)
                time.sleep(0.1)
                text = requests.get(url).text
                soup = BeautifulSoup(text, "lxml")
                # total_pic = soup.select_one("#contents_area_full > ul > ul")
                # list_pic = total_pic.findChildren("div", recursive=False)
                total_pic = soup.select_one("#contents_area_full > ul > ul")
                list_pic = total_pic.findChildren("li")
                if len(list_pic) == 0:
                    break
                for i, pic in enumerate(list_pic):
                    try:
                        thumb_png_get(pic)
                    except Exception:
                        # print(Exception.args)
                        pass
                    num += 1
                page += 1
            except:
                print("키워드 검색 끝")
                break


https://www.10000recipe.com/recipe/list.html?q=콩나물국밥&order=reco&page=1
https://recipe1.ezmember.co.kr/cache/recipe/2017/10/12/e0493587a0b6f093762c8393ecac50081_m.jpg /recipe/6877754
https://recipe1.ezmember.co.kr/cache/recipe/2018/12/10/35e82c5f7dddf837acfb92a4911fd7101_m.jpg /recipe/6901667
https://www.10000recipe.com/recipe/view.html?seq=6901667
cont_ingre2
https://recipe1.ezmember.co.kr/cache/recipe/2016/07/17/2ced014879bfa4a0c3b7ce042304b81d1_m.jpg /recipe/6852913
https://www.10000recipe.com/recipe/view.html?seq=6852913
cont_ingre2
https://recipe1.ezmember.co.kr/cache/recipe/2020/03/10/0b4f4d61a873a93b408fa3caea96d3c51_m.jpg /recipe/6928170
https://www.10000recipe.com/recipe/view.html?seq=6928170
cont_ingre2
https://recipe1.ezmember.co.kr/cache/recipe/2015/11/11/a87aab5b4a12eae319f4e20e4a409f231_m.jpg /recipe/6837942
https://www.10000recipe.com/recipe/view.html?seq=6837942
cont_ingre2
https://recipe1.ezmember.co.kr/cache/recipe/2016/05/21/c5508465f267385f65c9d9ef7df91c931_m.jpg /re